In [1]:
import cv2
import os
import time
import pymavlink
from multiprocessing import Process
import numpy as np

In [2]:
#Loading training images and labels and saving the data
images = []
labels = np.array([2,0,0,3,3,9,9,6,6,8,8,6,6,1,1,9,9,6,6,5,5,8,8,7,7,7,7,5,5,7,7,1,1,3,3,4,4,2,2])
trainedlabels = labels[:,np.newaxis]
#print(trainedlabels.shape)
#print(trainedlabels)
picnum = 39
img_data = []
for i in range(picnum):
	tempimg = cv2.imread('../images/%1d.png' %i)
	tempimg = cv2.cvtColor(tempimg, cv2.COLOR_BGR2GRAY)
	#tempimg = cv2.resize(tempimg, (10,10), interpolation=cv2.INTER_AREA)
	#cv2.waitKey(500)
	#print('%1d.png' %i)
	#cv2.imshow('boo', tempimg)
	images.append(tempimg.copy())
	#print(tempimg)
	img_data.append(tempimg.copy())
images = np.array(images)
img_data = np.array(img_data)
#print(img_data)
#train_array = img_data.reshape(-1,100).astype(np.float32)
train_array = img_data.reshape(-1,900).astype(np.float32)
#print(train_array.shape)
#print(train_array[1])
#cv2.waitKey(5000)
#cv2.destroyAllWindows()
np.savez('knn_data_test.npz',train_array=train_array, trainedlabels=trainedlabels)


In [3]:

with np.load('knn_data_test.npz') as data:
    print( data.files )
    train_array = data['train_array']
    trainedlabels = data['trainedlabels']
#print(trainedlabels)
knn = cv2.ml.KNearest_create()
knn.train(train_array, cv2.ml.ROW_SAMPLE, trainedlabels)

['train_array', 'trainedlabels']


True

In [4]:
ret,result,neighbours,dist = knn.findNearest(train_array,k=1)
matches = result==trainedlabels
correct = np.count_nonzero(matches)
accuracy = correct*100.0/result.size
print( accuracy )
#print(int(result[2]))

100.0


In [5]:
#Native 1920x1080, video test 960x544
# left top corner (x,y), right top corner(x,y), right bottom corner(x,y), left bottom corner(x,y)
x_res = 960
y_res = 544

lat_contour = np.array([[int(730/960*x_res),int(326/544*y_res)],[int(920/960*x_res),int(326/544*y_res)],[int(920/960*x_res),int(354/544*y_res)],[int(730/960*x_res),int(354/544*y_res)]]) 
lon_contour = np.array([[int(730/960*x_res),int(355/544*y_res)],[int(920/960*x_res),int(355/544*y_res)],[int(920/960*x_res),int(380/544*y_res)],[int(730/960*x_res),int(380/544*y_res)]]) 
alt_contour = np.array([[int(20/960*x_res),int(410/544*y_res)],[int(110/960*x_res),int(410/544*y_res)],[int(110/960*x_res),int(440/544*y_res)],[int(20/960*x_res),int(440/544*y_res)]]) 
contour_arr = np.array([lat_contour, lon_contour, alt_contour])
lat_width = abs(lat_contour[0,0]-lat_contour[1,0])
lat_height = abs(lat_contour[0,1]-lat_contour[2,1])
lon_width = abs(lon_contour[0,0]-lon_contour[1,0])
lon_height = abs(lon_contour[0,1]-lon_contour[2,1])
alt_width = abs(alt_contour[0,0]-alt_contour[1,0])
alt_height = abs(alt_contour[0,1]-alt_contour[2,1])
#print(np.shape(contour_arr))

In [6]:
def img_to_number(img_frame, img_name):
	number_contour = cv2.findContours(img_frame.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
	number_contour = number_contour[0]
	#print(number_contour)
	good_contour = []
	good_bounding_boxes = []
	img_array = []
	img_array_resized = []
	#resize_size = (10,10)
	#resize_size = (20,20)
	resize_size = (30,30)
	for i in number_contour:
		(x,y,w,h) = cv2.boundingRect(i)
		#print(x,y,w,y,i,cv2.boundingRect(i))
		if w>=5 and (h>= 15):
			good_contour.append(i)
			good_bounding_boxes.append(cv2.boundingRect(i))
	#good_bounding_boxes = list(good_bounding_boxes)
	good_bounding_boxes_arr = np.array(good_bounding_boxes)
	#print(good_bounding_boxes_arr)
	#print(type(good_bounding_boxes_arr))
	#print(good_bounding_boxes_arr[1])
	#print(type(good_bounding_boxes_arr[1]))
	#print(int(good_bounding_boxes_arr[1][0]))
	#print(type((good_bounding_boxes_arr[0][1])))
	i = 0
	for i in range(good_bounding_boxes_arr.shape[0]):
		y1 = good_bounding_boxes_arr[i,1].item()
		y2 = int(good_bounding_boxes_arr[i,1])+int(good_bounding_boxes_arr[i,3])
		x1 = int(good_bounding_boxes_arr[i,0])
		x2 = int(good_bounding_boxes_arr[i,0])+int(good_bounding_boxes_arr[i,2])
		tempimg = img_frame[y1:y2, x1:x2]
		img_array.append(tempimg.copy())
		temp_resized_img = cv2.resize(tempimg, resize_size, interpolation=cv2.INTER_AREA)
		img_array_resized.append(temp_resized_img.copy())
		#cv2.imwrite('numberr%5d.png' %i, temp_resized_img.copy())
	#img_frame = cv2.drawContours(img_frame, good_contour, -1, (255,255,255), 2)
	#cv2.imshow(img_name, img_frame)
	#numarea = img_frame[5:5+19, 173:173+12]
	#cv2.imshow('bam', img_array_resized[2])
	img_array_resized = np.array(img_array_resized)
	#img_array_resized = img_array_resized.reshape(-1,100).astype(np.float32)
	img_array_resized = img_array_resized.reshape(-1,900).astype(np.float32)
	ret,result,neighbours,dist = knn.findNearest(img_array_resized,k=1)
	endstring = str(int(result[(len(result)-1)]))
	i=0
	for i in range (len(result)-1):
		endstring = endstring + str(int(result[(len(result)-2-i)]))
	if(len(result)>=7):
		endnum = int(endstring)/(10000000)
	else:
		endnum = int(endstring) 
	#print(endnum)
	#print(len(result))
	#cv2.imshow('nam', numarea)
	return endnum

In [7]:
#Displays every 20th frame from video currently, live feed later
testvid = cv2.VideoCapture('C:/Users/Eriks/Desktop/drone_feed_test.mp4') 
videofps = 60
capture_frequency = 5 # analyzed frames per second
i = 0
show_full_frame = 0 #Value 1 used for seeing the full screen before area cropping
while(testvid.isOpened()):
	for i in range (int(videofps/capture_frequency)):
		framestatus, frame = testvid.read()
	i = 0
	if not framestatus:
		print('boowomp')
		break
	grayframe = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
	framestatus2, tresholdframe = cv2.threshold(grayframe,220,255,cv2.THRESH_TOZERO) #Removes background
	if not framestatus2:
		print('boowomp')
		break
	if(show_full_frame == 1): #Testing the full view
		cv2.drawContours(tresholdframe, contour_arr, -1 ,(255,255,255), 1)
		cv2.imshow('frame', tresholdframe)
	lat_area = tresholdframe[lat_contour[0,1]:lat_contour[0,1]+lat_height, lat_contour[0,0]:lat_contour[0,0]+lat_width]
	lon_area = tresholdframe[lon_contour[0,1]:lon_contour[0,1]+lon_height, lon_contour[0,0]:lon_contour[0,0]+lon_width]
	alt_area = tresholdframe[alt_contour[0,1]:alt_contour[0,1]+alt_height, alt_contour[0,0]:alt_contour[0,0]+alt_width]
	#cv2.imshow('lat', lat_area)
	
	lat = img_to_number(lat_area, 'lat')
	lon = img_to_number(lon_area, 'lon')
	alt = img_to_number(alt_area, 'alt')
	print(lat,lon,alt)
	#cv2.imshow('lon', lon_area)
	#cv2.imshow('alt', alt_area)
	#cv2.imwrite('drone_test_frame_lat.png', lat_area)
	#print('show')
	if cv2.waitKey(1000) == ord('q'):
		break
testvid.release()
cv2.destroyAllWindows()
print('done')

24.3175778 56.9168693 0
24.3175778 56.9168693 0
24.3175778 56.9168693 0
24.3175778 56.9168693 0
24.3175778 56.9168693 0
24.3175778 56.9168693 0
24.3175778 56.9168693 0
